<a href="https://colab.research.google.com/github/chrispi21/python-dataeng/blob/main/04_obsluga_plikow_i_manager_kontekstu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Przygotowanie do zajęć

Docs:
1. https://realpython.com/working-with-files-in-python/
2. https://docs.python.org/3/library/filesys.html
3. https://docs.python.org/3/library/io.html#io-overview
4. https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files


Ćwiczenia:
1. https://www.w3resource.com/python-exercises/file/

Zacznijmy od pobrania plików z danymi:

In [1]:
!wget -O pracownicy.csv https://raw.githubusercontent.com/chrispi21/python-dataeng/refs/heads/main/pracownicy.csv

--2025-03-07 13:14:44--  https://raw.githubusercontent.com/chrispi21/python-dataeng/refs/heads/main/pracownicy.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1526 (1.5K) [text/plain]
Saving to: ‘pracownicy.csv’

pracownicy.csv      100%[===================>]   1.49K  --.-KB/s    in 0s      

2025-03-07 13:14:45 (17.3 MB/s) - ‘pracownicy.csv’ saved [1526/1526]



`!` służy do wykonywania poleceń powłoki systemowej (ang. `shell`).

Sprawdźmy, czy udało się pobieranie naszego pliku i gdzie się znajduje:

In [2]:
!ls -la pracownicy.csv

-rw-r--r-- 1 root root 1526 Mar  7 13:14 pracownicy.csv


In [3]:
!pwd

/content


# Standardowe podejście

Otwórzmy plik:

In [27]:
sciezka = "/content/pracownicy.csv"
plik = open(sciezka)

Odczytajmy dane:

In [28]:
dane = plik.readlines()

Dane są w postaci niesparsowanej:

In [29]:
dane

['ID,Imię,Nazwisko,Stanowisko,Departament,Staż pracy (lata)\n',
 '1,Piotr,Kowalski,Inżynier produkcji,Marketing,21\n',
 '2,Ewa,Wiśniewska,Kierownik projektu,Finanse,23\n',
 '3,Jan,Kowalski,Kierownik projektu,HR,5\n',
 '4,Piotr,Lewandowski,Analityk finansowy,Marketing,12\n',
 '5,Piotr,Kamiński,Analityk finansowy,Marketing,23\n',
 '6,Adam,Kowalski,Kierownik projektu,Produkcja,29\n',
 '7,Adam,Szymański,Analityk finansowy,Zarządzanie,19\n',
 '8,Adam,Nowak,Marketing Specialist,HR,15\n',
 '9,Anna,Lewandowski,Inżynier produkcji,Zarządzanie,7\n',
 '10,Maria,Woźniak,Kierownik projektu,HR,30\n',
 '11,Maria,Szymańska,HR Manager,Marketing,12\n',
 '12,Tomasz,Wójcik,Inżynier produkcji,Marketing,29\n',
 '13,Adam,Dąbrowski,Marketing Specialist,HR,14\n',
 '14,Marek,Zieliński,Specjalista ds. IT,IT,13\n',
 '15,Agnieszka,Zielińska,Marketing Specialist,Marketing,25\n',
 '16,Marek,Zieliński,Marketing Specialist,HR,3\n',
 '17,Katarzyna,Nowak,Marketing Specialist,HR,20\n',
 '18,Piotr,Szymański,Inżynier produk

Zamknijmy plik:

In [30]:
plik.close()

Wady powyższego podejścia:

* Konieczność parsowania danych
* Trzeba zamknąć plik
* Musimy zadbać o obsługę wyjątków i wymusić zamknięcie pliku

Domyślnie pliki otwierane są w następującym trybie:
1. Do odczytu (`read mode`). Można zmienić na `write` albo otworzyć w trybie zapis i odczyt.
2. Tekstowym (domyślnie kodowanie zgodne z platformą). Można zmienić kodowanie. Można zmienić na tryb binarny.

Później zajmiemy się zapisem danych - spróbujmy znaleźć obejścia dla problemów.

# Manager kontekstu (ang. `context manager`)

Rozwiążemy problem pamiętania o zamknięciu pliku.

Docs:
1. https://book.pythontips.com/en/latest/context_managers.html
2. https://realpython.com/python-with-statement/#managing-resources-in-python

Dla chętnych:
1. https://docs.python.org/3/library/contextlib.html + https://realpython.com/python-with-statement/#creating-function-based-context-managers


In [20]:
with open(sciezka) as plik:
  dane_v2 = plik.readlines()

Manager kontekstu pozwala na bezpieczne otwarcie i zamknięcie pliku (oraz innych zasobów).

Możemy wyświetlić dane:

In [21]:
dane_v2

['ID,Imię,Nazwisko,Stanowisko,Departament,Staż pracy (lata)\n',
 '1,Piotr,Kowalski,Inżynier produkcji,Marketing,21\n',
 '2,Ewa,Wiśniewska,Kierownik projektu,Finanse,23\n',
 '3,Jan,Kowalski,Kierownik projektu,HR,5\n',
 '4,Piotr,Lewandowski,Analityk finansowy,Marketing,12\n',
 '5,Piotr,Kamiński,Analityk finansowy,Marketing,23\n',
 '6,Adam,Kowalski,Kierownik projektu,Produkcja,29\n',
 '7,Adam,Szymański,Analityk finansowy,Zarządzanie,19\n',
 '8,Adam,Nowak,Marketing Specialist,HR,15\n',
 '9,Anna,Lewandowski,Inżynier produkcji,Zarządzanie,7\n',
 '10,Maria,Woźniak,Kierownik projektu,HR,30\n',
 '11,Maria,Szymańska,HR Manager,Marketing,12\n',
 '12,Tomasz,Wójcik,Inżynier produkcji,Marketing,29\n',
 '13,Adam,Dąbrowski,Marketing Specialist,HR,14\n',
 '14,Marek,Zieliński,Specjalista ds. IT,IT,13\n',
 '15,Agnieszka,Zielińska,Marketing Specialist,Marketing,25\n',
 '16,Marek,Zieliński,Marketing Specialist,HR,3\n',
 '17,Katarzyna,Nowak,Marketing Specialist,HR,20\n',
 '18,Piotr,Szymański,Inżynier produk

Nie możemy już odczytać zamkniętego pliku:

In [23]:
plik.readlines()

ValueError: I/O operation on closed file.

Ćwiczenie

Co się stanie w przypadku błędu?

In [32]:
with open(sciezka) as plik2:
  # Wymuszamy błąd
  1 / 0
  plik2.readlines()

ZeroDivisionError: division by zero

In [33]:
# czy mogę odczytać dabe
plik2.readlines()

ValueError: I/O operation on closed file.

A teraz?

In [34]:
plik3 = open(sciezka)
1 / 0
plik3.readlines()

ZeroDivisionError: division by zero

In [35]:
plik3.readlines()

['ID,Imię,Nazwisko,Stanowisko,Departament,Staż pracy (lata)\n',
 '1,Piotr,Kowalski,Inżynier produkcji,Marketing,21\n',
 '2,Ewa,Wiśniewska,Kierownik projektu,Finanse,23\n',
 '3,Jan,Kowalski,Kierownik projektu,HR,5\n',
 '4,Piotr,Lewandowski,Analityk finansowy,Marketing,12\n',
 '5,Piotr,Kamiński,Analityk finansowy,Marketing,23\n',
 '6,Adam,Kowalski,Kierownik projektu,Produkcja,29\n',
 '7,Adam,Szymański,Analityk finansowy,Zarządzanie,19\n',
 '8,Adam,Nowak,Marketing Specialist,HR,15\n',
 '9,Anna,Lewandowski,Inżynier produkcji,Zarządzanie,7\n',
 '10,Maria,Woźniak,Kierownik projektu,HR,30\n',
 '11,Maria,Szymańska,HR Manager,Marketing,12\n',
 '12,Tomasz,Wójcik,Inżynier produkcji,Marketing,29\n',
 '13,Adam,Dąbrowski,Marketing Specialist,HR,14\n',
 '14,Marek,Zieliński,Specjalista ds. IT,IT,13\n',
 '15,Agnieszka,Zielińska,Marketing Specialist,Marketing,25\n',
 '16,Marek,Zieliński,Marketing Specialist,HR,3\n',
 '17,Katarzyna,Nowak,Marketing Specialist,HR,20\n',
 '18,Piotr,Szymański,Inżynier produk

In [36]:
plik3.close()

Ćwiczenie dla chętnych

Zapoznaj się z:
1. https://realpython.com/python-with-statement/#creating-function-based-context-managers
2. https://docs.python.org/3/library/os.html#os.environ

Utwórz manager kontekstu, który będzie inicjalizował zmienne środowiskowe a następnie je czyścił korzystając z `contextlib.contextmanager`.

Dla uproszczenie nie przejmujemy się obecnym stanem zmiennych środowiskowych i przywracaniem ich pierwotnych wartości.

Przykład:
```python
# powinno zadziałać wyświetlając kolejno:
# moja zmienna 1
# moja_zmienna_2
with env_var(MY_ENV_1="moja zmienna 1", MY_ENV_2="moja_zmienna_2"):
  print(os.environ["MY_ENV_1"])
  print(os.environ["MY_ENV_2"])

# powinno zakończyć się błędem:
print(os.environ["MY_ENV_1"])

```

In [ ]:
# @title Rozwiazanie

In [41]:
# @title Podpowiedź

import os
import contextlib

@contextlib.contextmanager
def env_var(**kwargs):
  os.environ.update(kwargs)
  yield
  for k in kwargs.keys():
    del os.environ[k]

# powinno zadziałać
with env_var(MY_ENV_1="moja zmienna 1", MY_ENV_2="moja_zmienna_2"):
  print(os.environ["MY_ENV_1"])
  print(os.environ["MY_ENV_2"])

# powinno zakończyć się błędem:
print(os.environ["MY_ENV_1"])

moja zmienna 1
moja_zmienna_2


KeyError: 'MY_ENV_1'

Inne zastosowania:
1. Obsługa połączeń (np. do baz danych)
2. Obsługa transakcji w bazach danych
3. Obsługa plików temporalnych i innych zasobów tymczasowych
4. Obsługa innych zasobów, dla których wymagane jest obsłużenie zamknięcia zasobu

Do tworzenie własnych managerów kontekstu można wykorzystać bibiotekę `contextlib` ([link](https://docs.python.org/3/library/contextlib.html)). Posiada ona też wiele gotowych managerów kontekstu.

# Odczyt *csv*

Rozwiążemy teraz problem samodzielnego parsowania zawartości pliku *csv*

In [ ]:
import csv

In [ ]:
sciezka = "/content/pracownicy.csv"
with open(sciezka,  newline='') as plik_csv:
  dane_csv = list(csv.reader(plik_csv, delimiter=","))

In [ ]:
dane_csv

[['ID', 'Imię', 'Nazwisko', 'Stanowisko', 'Departament', 'Staż pracy (lata)'],
 ['1', 'Piotr', 'Kowalski', 'Inżynier produkcji', 'Marketing', '21'],
 ['2', 'Ewa', 'Wiśniewska', 'Kierownik projektu', 'Finanse', '23'],
 ['3', 'Jan', 'Kowalski', 'Kierownik projektu', 'HR', '5'],
 ['4', 'Piotr', 'Lewandowski', 'Analityk finansowy', 'Marketing', '12'],
 ['5', 'Piotr', 'Kamiński', 'Analityk finansowy', 'Marketing', '23'],
 ['6', 'Adam', 'Kowalski', 'Kierownik projektu', 'Produkcja', '29'],
 ['7', 'Adam', 'Szymański', 'Analityk finansowy', 'Zarządzanie', '19'],
 ['8', 'Adam', 'Nowak', 'Marketing Specialist', 'HR', '15'],
 ['9', 'Anna', 'Lewandowski', 'Inżynier produkcji', 'Zarządzanie', '7'],
 ['10', 'Maria', 'Woźniak', 'Kierownik projektu', 'HR', '30'],
 ['11', 'Maria', 'Szymańska', 'HR Manager', 'Marketing', '12'],
 ['12', 'Tomasz', 'Wójcik', 'Inżynier produkcji', 'Marketing', '29'],
 ['13', 'Adam', 'Dąbrowski', 'Marketing Specialist', 'HR', '14'],
 ['14', 'Marek', 'Zieliński', 'Specjalista

# Odczyt za pomocą generatora